In [236]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn import datasets
from numpy import array
from pandas import DataFrame
from pandas import concat
from keras.models import model_from_json
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder


In [237]:
def rmse(objetivo, estimaciones):
    return np.sqrt(metrics.mean_squared_error(objetivo, estimaciones)
                  )

In [238]:
def  ajustested_r2(objetivo, estimaciones,n,k):
    r2=metrics.r2_score(objetico,estimaciones)
    return 1-(1-r2)*(n-1)/(n-k-1)

In [239]:
def evaluar_modelo(objetivo,estimaciones,n,k):
    return {
        "rmse":rmse(objetivo,estimaciones),
        "mae": metrics.mean_absolute_error(objetivo,estimaciones),
        "ajustested_r2": ajustested_r2(objetivo,estimaciones,n,k)
    }

In [240]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
# observaciones pasadas  (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
# tiempos futuros  (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    agg = concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [220]:
#Cargo el modelo antes generado.
json_file = open('../Modelamiento/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# Cargo el nuevo modelo
loaded_model.load_weights("../Modelamiento/LTSM_Cali.h5")
loaded_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 50)                12800     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 51        
Total params: 12,851
Trainable params: 12,851
Non-trainable params: 0
_________________________________________________________________


In [254]:
#Prueba manual con los datos 
#PM10
#DirVien
#H2S
#Humedad
#Lluvia
#NO2
#O3
#CO
#RSolar
#SO2
#Tasa_crecimiento_general
#Temp
#VelVien
x_input = np.array([20,148.6000061,0,72.69999695,1.269999981,0,17.64,2851.524414,
                    302.7999878,12.91359997,0.097400002,24.89999962,1.299999952])
print(x_input.shape)

df = pd.DataFrame({ 'PM10':    [20,20],
                    'DirVien': [148.6000061,148.6000061],
                    'H2S': [0,0],
                    'Humedad': [72.69999695,72.69999695],
                    'Lluvia': [1.269999981,1.269999981],
                    'NO2': [0,0],
                    'O3': [17.64,17.64],
                    'CO': [2851.524414,2851.524414],
                    'RSolar': [302.7999878,302.7999878],
                    'SO2': [12.91359997,12.91359997],
                    'Tasa_crecimiento_general': [0.90,0.90],
                    'Temp': [24.89999962,24.89999962],
                    'VelVien': [1.299999952,1.299999952]
                    
                   })



(13,)


In [242]:
df

,PM10,DirVien,H2S,Humedad,Lluvia,NO2,O3,CO,RSolar,SO2,Tasa_crecimiento_general,Temp,VelVien
0,20,148.600006,0,72.699997,1.27,0,17.64,2851.524414,302.799988,12.9136,0.9,24.9,1.3
1,20,148.600006,0,72.699997,1.27,0,17.64,2851.524414,302.799988,12.9136,0.9,24.9,1.3


In [243]:
print(df.values)

[[2.00000000e+01 1.48600006e+02 0.00000000e+00 7.26999969e+01
  1.26999998e+00 0.00000000e+00 1.76400000e+01 2.85152441e+03
  3.02799988e+02 1.29136000e+01 9.00000000e-01 2.48999996e+01
  1.29999995e+00]
 [2.00000000e+01 1.48600006e+02 0.00000000e+00 7.26999969e+01
  1.26999998e+00 0.00000000e+00 1.76400000e+01 2.85152441e+03
  3.02799988e+02 1.29136000e+01 9.00000000e-01 2.48999996e+01
  1.29999995e+00]]


In [244]:
df.values

array([[2.00000000e+01, 1.48600006e+02, 0.00000000e+00, 7.26999969e+01,
        1.26999998e+00, 0.00000000e+00, 1.76400000e+01, 2.85152441e+03,
        3.02799988e+02, 1.29136000e+01, 9.00000000e-01, 2.48999996e+01,
        1.29999995e+00],
       [2.00000000e+01, 1.48600006e+02, 0.00000000e+00, 7.26999969e+01,
        1.26999998e+00, 0.00000000e+00, 1.76400000e+01, 2.85152441e+03,
        3.02799988e+02, 1.29136000e+01, 9.00000000e-01, 2.48999996e+01,
        1.29999995e+00]])

In [255]:
values = df.values
values = values.reshape((values.shape[0], 1, values.shape[1]))
yhat = loaded_model.predict(values, verbose=0)
print(yhat)

[[-1.7533857]
 [-1.7533857]]


In [246]:
#Prueba de modelo, cargo la informacion de la ERA 2010 crecimiento 0.9

dataOriginal = pd.read_excel('../PreparacionDatos/eraDatos.xlsx' , sheet_name='eraDatos' ,header=0)
dataOriginal = dataOriginal.drop(dataOriginal.columns[[0]], axis=1) 
dataOriginal.head(5)

,PM10,DirVien,H2S,Humedad,Lluvia,NO2,O3,CO,RSolar,SO2,Tasa_crecimiento_general,Temp,VelVien
0,0.0,267.799988,0,59.799999,0.0,0.0,17.64,1101.839844,0.0,6.0218,0.0974,28.100000,3.4
1,0.0,277.299988,0,60.099998,0.0,0.0,17.64,1018.621826,0.0,6.0893,0.0974,27.900000,3.0
2,0.0,271.799988,0,61.700001,0.0,0.0,17.64,763.458923,0.0,4.5507,0.0974,27.700001,2.2
3,0.0,269.299988,0,64.300003,0.0,0.0,17.64,635.792908,0.0,3.9924,0.0974,27.200001,1.6
4,0.0,39.700001,0,71.000000,0.0,0.0,17.64,588.573120,0.0,3.3259,0.0974,25.700001,0.5


In [247]:
x_input = np.array(dataOriginal)
x_input.shape

(7616, 13)

In [248]:
#Remodelar la entrada para que sea 3D [muestras, pasos de tiempo, características]
x_input = x_input.reshape((x_input.shape[0], 1, x_input.shape[1]))
print(x_input.shape)

(7616, 1, 13)


In [249]:
yhat_original = loaded_model.predict(x_input, verbose=0)
print(yhat_original)

[[-1.2617254 ]
 [-1.2618513 ]
 [-1.6274126 ]
 ...
 [-1.8195732 ]
 [ 0.5457126 ]
 [ 0.20386183]]


In [250]:
#Cambio el modelo a 93% de contaminacion.

data2028 = pd.read_excel('../PreparacionDatos/eraDatos.xlsx' , sheet_name='eraDatos' ,header=0)
data2028 = data2028.drop(data2028.columns[[0]], axis=1) 
data2028['Tasa_crecimiento_general']=0.93
data2028.head(5)

,PM10,DirVien,H2S,Humedad,Lluvia,NO2,O3,CO,RSolar,SO2,Tasa_crecimiento_general,Temp,VelVien
0,0.0,267.799988,0,59.799999,0.0,0.0,17.64,1101.839844,0.0,6.0218,0.93,28.100000,3.4
1,0.0,277.299988,0,60.099998,0.0,0.0,17.64,1018.621826,0.0,6.0893,0.93,27.900000,3.0
2,0.0,271.799988,0,61.700001,0.0,0.0,17.64,763.458923,0.0,4.5507,0.93,27.700001,2.2
3,0.0,269.299988,0,64.300003,0.0,0.0,17.64,635.792908,0.0,3.9924,0.93,27.200001,1.6
4,0.0,39.700001,0,71.000000,0.0,0.0,17.64,588.573120,0.0,3.3259,0.93,25.700001,0.5


In [251]:
x_input_2028 = np.array(data2028)
x_input_2028 .shape
x_input_2028  = x_input_2028.reshape((x_input_2028.shape[0], 1, x_input_2028.shape[1]))
print(x_input.shape)

(7616, 1, 13)


In [252]:
yhat_2018 = loaded_model.predict(x_input, verbose=0)
print(yhat_2018)

[[-1.2617254 ]
 [-1.2618513 ]
 [-1.6274126 ]
 ...
 [-1.8195732 ]
 [ 0.5457126 ]
 [ 0.20386183]]


In [253]:
type(yhat_2018)

numpy.ndarray

In [256]:
dataset = pd.DataFrame({'yhat_2018': yhat_2018[:, 0], 'yhat_original': yhat_original[:, 0]})

In [257]:
dataset.to_excel("union.xlsx", sheet_name='union',encoding='UTF-8') #Se crea el archivo definitivo.